In [1]:
from typing import TypedDict
from langgraph.graph import StateGraph

# SIngle Input

In [2]:
class AgentState(TypedDict):
    """
    Represents the state of the agent.
    """
    message: str

In [3]:
a1 = AgentState(message="Hello, world!")

In [4]:
a1

{'message': 'Hello, world!'}

In [5]:
a2 = AgentState(message=1234)

In [6]:
a2

{'message': 1234}

In [7]:
def greeting_node(state: AgentState) -> AgentState:
    """
    A node that returns a greeting message.
    """
    state['message'] = "Hey " + state['message'] + ", how are you?"
    return state

In [8]:
greeting_node(state={'message': "Ajit"})

{'message': 'Hey Ajit, how are you?'}

In [9]:
graph = StateGraph(state_schema=AgentState)
graph

In [10]:
graph.add_node(node="greeting", action=greeting_node)

In [11]:
graph.set_entry_point(key="greeting")
graph.set_finish_point(key="greeting")

In [12]:
app = graph.compile()

In [ ]:
app

In [ ]:
app.get_graph()

In [ ]:
app.invoke(input={'message': "Ajit"})

In [ ]:
result = app.invoke(input={'message': "Ajit Emmanuel"})
result

# Multiple Input Graph

In [ ]:
from typing import TypedDict
from langgraph.graph import StateGraph

In [ ]:
class AgentState(TypedDict):
    """
    Represents the state of the agent.
    """
    values: list[int ]
    name: str
    result: str

In [ ]:
def process_values(state: AgentState) -> AgentState:
    """
    A node that processes a list of values.
    """
    state['result'] = f"Hi there {state['name']}! Your sum is {sum(state['values'])}."
    return state

In [ ]:
graph = StateGraph(state_schema=AgentState)
graph.add_node(node="processor", action=process_values)
graph.set_entry_point(key="processor")
graph.set_finish_point(key="processor")
app1 = graph.compile()
app1

In [ ]:
app1.invoke(input={'values': [1, 2, 3], 'name': "Ajit"})

# Sequential Graph

In [ ]:
class AgentState(TypedDict):
    """
    Represents the state of the agent.
    """
    name: str
    age: int
    final: str

In [ ]:
def first_node(state: AgentState) -> AgentState:
    """
    A node that processes the first part of the state.
    """
    state['final'] = f"Hello {state['name']}!"
    return state

def second_node(state: AgentState) -> AgentState:
    """
    A node that processes the second part of the state.
    """
    state['final'] += f" You are {state['age']} years old."
    return state

In [ ]:
graph = StateGraph(state_schema=AgentState)
graph.add_node(node="first", action=first_node)
graph.add_node(node="second", action=second_node)

graph.add_edge(start_key ="first", end_key ="second")

graph.set_entry_point(key="first")
graph.set_finish_point(key="second")

app2 = graph.compile()
app2

In [ ]:
app2.invoke(input={'name': "Ajit", 'age': 40})

In [ ]:
class AgentState(TypedDict):
    """
    Represents the state of the agent.
    """
    name: str
    age: int
    skills: list[str]